In [10]:
import pandas as pd
import numpy as np
from numpy import zeros

## DATA PRE-PROCESSING

In [11]:
# how many data ?
num_of_data = 2
data_load1 = pd.read_csv('ahc_complete_linkage.csv')
data_load2 = pd.read_csv('ahc_average_linkage.csv')
# data.columns = ["sepal length", "sepal width", "petal length", "petal width","label"]
# data_set = data[data.columns.difference(['label'])]
data_fr1 = pd.DataFrame(columns=['sepal length','sepal width','petal length','petal width','cluster'], data=data_load1[['sepal length','sepal width','petal length','petal width','cluster']])
data_fr2 = pd.DataFrame(columns=['sepal length','sepal width','petal length','petal width','cluster'], data=data_load2[['sepal length','sepal width','petal length','petal width','cluster']])
data = {}
data[0] = data_fr1
data[1] = data_fr2

## CALCULATE PROCESS

In [12]:
def get_avg_cluster(data_cluster):
    avg_sepal_length = data_cluster['sepal length'].mean()
    avg_sepal_width = data_cluster['sepal width'].mean()
    avg_petal_length = data_cluster['petal length'].mean()
    avg_petal_width = data_cluster['petal width'].mean()
    data_avg = np.array([avg_sepal_length,avg_sepal_width,avg_petal_length,avg_petal_width])
    return data_avg;
def sikmavc( d,avg_cluster ):
    a = float(d['sepal length'] - avg_cluster[0])
    b = float(d['sepal width'] - avg_cluster[1])
    c = float(d['petal length'] - avg_cluster[2])
    d = float(d['petal width'] - avg_cluster[3])
    new_data = np.array([a,b,c,d])
    final_data = np.dot(new_data,new_data)
    return final_data;
def vc(data_cluster):
    i = 0
    temp_res=0
#     data_cluster = data[data['cluster']==cluster_name]
    avg_d = get_avg_cluster(data_cluster) 
    nc = len(data_cluster)
    while i < nc:
        temp_res+=sikmavc(data_cluster.iloc[[i]],avg_d)
        i+=1
    vc_data = float((1/(nc-1)) * temp_res) 
    return vc_data;
iterat = 0
# loop process
vw = zeros([2,1])
vb = zeros([2,1])
v = zeros([2,1])
jenis = np.array(['ahc-complete-linkage','ahc-average-linkage'])
while iterat < num_of_data:
#     sikmavw=0
#     sikmavb=0
    # ========= calculate vw =============    
    # list cluster
    list_cluster = np.array([0,1,2])
    vc_cluster = zeros([len(list_cluster),1])
    k = 3
    j=0
    t=0
    sikmavw=0
    # calculate vc per cluster
    while j < k:
        vc_cluster[j] = vc(data[iterat][data[iterat]['cluster']==list_cluster[j]])
        j+=1
    while t < k:
        data_clus = data[iterat][data[iterat]['cluster']==list_cluster[t]]
        nc = len(data_clus)
        sikmavw+=(nc-1)*vc_cluster[t]
        t+=1
    N = len(data[iterat])
    vw[iterat]=float((1/(N-k))*sikmavw)
    # ========= calculate vw =============

    # ========= calculate vb =============
    # average data in each cluster
    j=0
    avg_per_cluster = zeros([len(list_cluster),4])
    while j < k:
        data_cluster2 = data[iterat][data[iterat]['cluster']==list_cluster[j]]
        avg_per_cluster[j] = get_avg_cluster(data_cluster2)
        j+=1
    # average all cluster
    j=0
    average_cluster = zeros([1,4])
    while j < avg_per_cluster.shape[1]:
        average_cluster[0][j] = avg_per_cluster[:,[j]].mean()
        j+=1
    #  vb final calculate
    j=0
    sikmavb = 0
    while j < k:
        data_clus = data[iterat][data[iterat]['cluster']==list_cluster[j]]
        nc = len(data_clus)
        selisih = avg_per_cluster[j] - average_cluster
        data_selisih = np.array([selisih[0][0],selisih[0][1],selisih[0][2],selisih[0][3]])
        vekt_selisih = np.dot(data_selisih,data_selisih)
        n_selisih = nc * vekt_selisih
        sikmavb+=n_selisih
        j+=1
    vb[iterat] = float((1/(k-1))*sikmavb)
    # ========= calculate vb =============

    # ========= calculate v =============
    v[iterat] = vw[iterat]/vb[iterat]
    iterat+=1

## RESULT

In [13]:
final = pd.DataFrame({'v':v.ravel(),'vb':vb.ravel(),'vw':vw.ravel(),'jenis':jenis.ravel()})
final

,jenis,v,vb,vw
0,ahc-complete-linkage,0.002026,300.840486,0.609622
1,ahc-average-linkage,0.001787,302.785773,0.541053
